In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, classification_report, multilabel_confusion_matrix
import matplotlib.pyplot as plt 
from matplotlib.patches import Patch
import seaborn as sns
%matplotlib inline
from IPython.display import display

In [ ]:
path = os.getcwd()
files_true = os.listdir(path+'/data_true')
files_pred = os.listdir(path+'/data_pred')

In [ ]:
df_true = pd.DataFrame()
for f in files_true:
    if f.endswith('.xlsx'):
        data = pd.read_excel('data_true/'+f,engine='openpyxl')
        df_true = df_true.append(data)     

In [ ]:
df_pred = pd.DataFrame()
for f in files_pred:
    if f.endswith('.xlsx')  and (f in files_true):
        data = pd.read_excel('data_pred/'+f,engine='openpyxl')
        df_pred = df_pred.append(data)   

### Exploration des données annotées manuellement

In [ ]:
df_true.head(50)

#### Nombre total d'entités annotées manuellement

In [ ]:
df_true.shape[0]

#### Occurrence des entités (true)

In [ ]:
df_true['Label'].value_counts()

In [ ]:
sns.set(font_scale=1.4)
df_true['Label'].value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
plt.xlabel("Label", labelpad=14)
plt.ylabel("Count of entities", labelpad=14)
plt.title("Count of entities by label", y=1.02);

In [ ]:
df_true['Entity'].mode()

#### Occurences des entités les plus fréquentes

In [ ]:
df_true.groupby(['Entity','Label']).size().sort_values(ascending=False).head(10)

#### Occurences des entités les plus fréquentes au sein de chaque label

In [ ]:
for item in df_true['Label'].unique():
    print('-------',item,'-------')
    display(df_true.loc[df_true['Label']==item].groupby(['Entity','Label']).size().sort_values(ascending=False).head(5))

### Exploration des données prédites par l'outil

In [ ]:
df_pred.head(20)

#### Nombre total d'entités prédites par l'outil

In [ ]:
df_pred.shape[0]

#### Occurrence des entités (pred)

In [ ]:
df_pred['Label'].value_counts()

In [ ]:
sns.set(font_scale=1.4)
df_pred['Label'].value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
plt.xlabel("Label", labelpad=14)
plt.ylabel("Count of entities", labelpad=14)
plt.title("Count of entities by label", y=1.02);

In [ ]:
df_pred['Entity'].mode()

#### Occurences des entités les plus fréquentes

In [ ]:
df_pred.groupby(['Entity','Label']).size().sort_values(ascending=False).head(10)

#### Occurences des entités les plus fréquentes au sein de chaque label

In [ ]:
for item in df_pred['Label'].unique():
    print('-------',item,'-------')
    display(df_pred.loc[df_pred['Label']==item].groupby(['Entity','Label']).size().sort_values(ascending=False).head(5))

### Exploration des données de l'évaluation

In [ ]:
df_pred=df_pred.drop(['Text'],axis =1,errors='ignore')

In [ ]:
df_true=df_true.drop(['Text'],axis =1,errors='ignore')
df_true = df_true[df_true['Entity'].notna()]

In [ ]:
df = pd.merge(df_true, df_pred, on=['Entity'], how='outer').drop_duplicates().fillna("O")

In [ ]:
df

In [ ]:
df.columns = ['Entity','y_true', 'y_pred']

In [ ]:
df.shape

In [ ]:
df.head(20)

In [ ]:
df.shape[0]

#### Exploration des données réelles

In [ ]:
df['y_true'].value_counts()

In [ ]:
sns.set(font_scale=1.4)
df['y_true'].value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
plt.xlabel("Label", labelpad=14)
plt.ylabel("Count of entities", labelpad=14)
plt.title("Count of entities by label (true)", y=1.02);

#### Les entités détectées par l'outil par erreur (Faux positifs)

In [ ]:
df.loc[df['y_true']=='O']

In [ ]:
print("Nombre total de Faux Positifs : ",len(df.loc[df['y_true']=='O']))

#### Exploration des données prédites

In [ ]:
df['y_pred'].value_counts()

In [ ]:
sns.set(font_scale=1.4)
df['y_pred'].value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
plt.xlabel("Label", labelpad=14)
plt.ylabel("Count of entities", labelpad=14)
plt.title("Count of entities by label (pred)", y=1.02);

#### Les entités annotées manuellement mais non détectées par l'outil (Faux négatifs)

In [ ]:
df.loc[df['y_pred']=='O']

In [ ]:
print("Nombre total de Faux Négatifs : ",len(df.loc[df['y_pred']=='O']))

### Erreurs détectées

In [ ]:
regex_list = ['TEL','PERIODE','EMAIL','LANG', 'URL', 'DATE']
regex_error_count = 0

In [ ]:
model_list = ['PER','LOC']
model_error_count = 0

In [ ]:
dict_error_regex = {}
dict_error_model = {}

In [ ]:
df_error = df.loc[(df['y_true']!=df['y_pred'])]

In [ ]:
print("Nombre d'erreurs détectées : ",len(df_error))

#### Erreurs détectées par label

In [ ]:
for item in df['y_true'].unique():
    if(((len(df_error.loc[df_error['y_true']==item])>0)|(len(df_error.loc[df_error['y_pred']==item])>0)) & (item!='O') ):      
        print('\n')
        print('-------',item,'-------')
        display(df_error.loc[(df_error['y_true']==item)|(df_error['y_pred']==item)])
        print('\n')
        if(item in regex_list):
            regex_error_count = regex_error_count+len(df_error.loc[(df_error['y_true']==item)|(df_error['y_pred']==item)])
            dict_error_regex[item] = len(df_error.loc[(df_error['y_true']==item)|(df_error['y_pred']==item)])
        else:
            model_error_count = model_error_count+len(df_error.loc[(df_error['y_true']==item)|(df_error['y_pred']==item)])
            dict_error_model[item] = len(df_error.loc[(df_error['y_true']==item)|(df_error['y_pred']==item)])

#### Sources des erreurs détectées

In [ ]:
plt.rcdefaults()

objects = ('Erreurs issues des regex', 'Erreurs issues du modèle')
y_pos = np.arange(len(objects))
performance = [regex_error_count,model_error_count]


plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Nombre d\'erreurs')
plt.title('Nombre d\'erreurs détectées')

plt.show()

In [ ]:
labels = 'Erreurs issues des regex', 'Erreurs issues du modèle'
sizes = [regex_error_count, model_error_count]
colors = ['lightcoral', 'lightskyblue']
explode = (0.1, 0)  # explode 1st slice

plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Pourcentage des erreurs détectées')
plt.axis('equal')
plt.show()

### Erreurs détectées par label

In [ ]:
nbr_regex_colors = ['lightcoral'] * len(dict_error_regex.values())
nbr_model_colors = ['lightskyblue'] * len(dict_error_model.values())
labels = list(dict_error_regex.keys())+list(dict_error_model.keys())
sizes = list(dict_error_regex.values())+list(dict_error_model.values())
colors = nbr_regex_colors+nbr_model_colors
explode = [0.1]* len(nbr_regex_colors+nbr_model_colors) 
plt.pie(sizes, labels=labels, colors=colors, explode=explode ,
autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Pourcentage des erreurs détectées par label')
plt.axis('equal')
plt.legend([
        Patch(facecolor='lightcoral'),
        Patch(facecolor='lightskyblue')
    ], ["Erreurs issues des regex", "Erreurs issues du modèle"], bbox_to_anchor=[1.5, 1], 
           loc='upper right', ncol=1)
plt.show()

In [ ]:
nbr_regex_errors = ['regex'] * len(dict_error_regex.values())
nbr_model_errors = ['model'] * len(dict_error_model.values())
plotdata = pd.DataFrame({
    "pies": list(dict_error_regex.values())+list(dict_error_model.values()), 
    "type": nbr_regex_errors+nbr_model_errors
    }, 
    index=list(dict_error_regex.keys())+list(dict_error_model.keys())
)

colours = {"regex": "lightcoral", "model": "lightskyblue"}
plotdata['pies'].plot(
        kind="bar", color=plotdata['type'].replace(colours)
).legend(
    [
        Patch(facecolor=colours['regex']),
        Patch(facecolor=colours['model'])
    ], ["Erreurs issues des regex", "Erreurs issues du modèle"], bbox_to_anchor=[1.5, 1], 
           loc='upper right', ncol=1
)
plt.title("Nombre d'erreurs détectées par label")
plt.xlabel("Labels")
plt.ylabel("Nombre d'erreurs")
plt.show()